# Uvod v NumPy


## Understanding Data Types in Python



```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```


```python
# Python code
result = 0
for i in range(100):
    result += i
```


In [ ]:
# Python code
x = 4
x = "four"



```C
/* C code */
int x = 4;
x = "four";  // FAILS
```


### A Python Integer Is More Than Just an Integer



```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```


<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/cint_vs_pyint.png" alt="Integer Memory Layout">

### A Python List Is More Than Just a List




In [ ]:
L = list(range(10))
L

In [ ]:
type(L[0])

In [ ]:
L2 = [str(c) for c in L]

In [ ]:
type(L2[0])

In [ ]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]


<img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/array_vs_list.png" alt="Array Memory Layout">

### Fixed-Type Arrays in Python



In [ ]:
import array

L = list(range(10))
A = array.array('i', L)
A

## Example: Data analysis in pure Python

In [ ]:
import csv

dataset_path = "data/f500_small.csv"



## How Vectorization Makes Code Faster



<p><img alt="Translating Python code to bytecode" src="https://s3.amazonaws.com/dq-content/289/bytecode.svg"></p>


<table>
<thead>
<tr>
<th>Language Type</th>
<th>Example</th>
<th>Time taken to write program</th>
<th>Control over program performance</th>
</tr>
</thead>
<tbody>
<tr>
<td>High-Level</td>
<td>Python</td>
<td>Low</td>
<td>Low</td>
</tr>
<tr>
<td>Low-Level</td>
<td>C</td>
<td>High</td>
<td>High</td>
</tr>
</tbody>
</table>



<p><img alt="For loop to sum rows" src="https://s3.amazonaws.com/dq-content/289/for_loop.svg"></p>

In [ ]:
my_numbers = [[6,5], [1,3], [5,6]]

sums = []

for row in my_numbers:
    row_sum = row[0] + row[1]
    sums.append(row_sum)
    
print(sums)    

<p><img src="./images/numpy_for.gif"></p>

<p><img src="./images/numpy_vectorized.gif"></p>

Primerjeva hitrosti:

In [ ]:
import numpy as np
size = 5_000_000

# Python - declaring lists
list1 = [i for i in range(size)]
list2 = [i for i in range(size)]

# Numpy - declaring arrays
array1 = np.arange(size)
array2 = np.arange(size)

In [ ]:
print(list1[:5])
print(array1[:5])

In [ ]:
%%timeit -n 3 -r 1

sums = []

for el1, el2 in zip(list1, list2):
    row_sum = el1 + el2
    sums.append(row_sum)

print(sums[:5])

In [ ]:
%%timeit -n 10 -r 1

sums = array1 + array2

## Numpy library

[Dokumentacija](http://www.numpy.org/)

In [ ]:
import numpy as np 

## Introduction to Ndarrays

<img alt="Dimensional Arrays" src="./images/one_dim.svg">

In [ ]:
data_ndarray = np.array([5, 10, 15, 20])

In [ ]:
#preverimo kakšen tip je spremenljivka
print(type(data_ndarray)) 

<img alt="Dimensional Arrays" src="./images/Two_Dim.svg">

In [ ]:
data2 = [[1, 2, 3, 4], [5, 6, 7, 8]]
arr2 = np.array(data2)
print(arr2)

## Reading CSV files with NumPy

Below is information about selected columns from the data set:
- `rank`
- `revenue`
- `revenue_change`
- `profits`
- `asset`
- `profit_change`




    np.genfromtxt(filename, delimiter=None)

- `filename`: A positional argument, usually a string representing the path to the text file to be read.
- `delimiter`: A named argument, specifying the string used to separate each value.
- `usecols`: Which columns to read, with 0 being the first. For example, usecols = (1, 4, 5) will extract the 2nd, 5th and 6th columns.



    data = np.genfromtxt('data.csv', delimiter=',')

In [ ]:
f500 = np.genfromtxt('data/f500_small.csv', delimiter=',', usecols=(1,2,3,4,5,6), skip_header=1)
f500_shape = f500.shape
f500_shape

In [ ]:
print(f500[:5])

## Array Shapes

In [ ]:
data_ndarray = np.array([[5, 10, 15], 
                         [20, 25, 30]])

In [ ]:
#the total number of elements of the array.
print(data_ndarray.size)

In [ ]:
#the number of axes (dimensions) of the array.
print(data_ndarray.ndim)

In [ ]:
# the size in bytes of each element of the array
print(data_ndarray.itemsize)

In [ ]:
print(data_ndarray.nbytes)

In [ ]:
# an object describing the type of the elements in the array. 
# One can create or specify dtype’s using standard Python types. 
# Additionally NumPy provides types of its own. numpy.int32, numpy.int16, and numpy.float64 are some examples.
print(data_ndarray.dtype)

<div class="alert alert-block alert-info">
<b>Vaja:</b> Preverite še vse lastnosti za f500 array.</div>

## Selecting and Slicing Rows and Items from ndarrays

<img alt="Dimensional Arrays" src="./images/selection_rows.svg">

    ndarray[row_index,column_index]

    # or if you want to select all
    # columns for a given set of rows
    ndarray[row_index]

<img alt="Dimensional Arrays" src="./images/selection_item.svg">

In [ ]:
# Create a 5x5 array of random integers in the interval [0, 10)
test = np.random.randint(0, 10, (5, 5))

<div class="alert alert-block alert-info">
<b>Vaja:</b> Izberite vrednost in vrstico f500 array-ja.</div>

## Selecting Columns and Custom Slicing ndarrays

<img alt="Dimensional Arrays" src="./images/selection_columns_updated.svg">

<img alt="Dimensional Arrays" src="./images/selection_1darray_updated.svg">

<img alt="Dimensional Arrays" src="./images/selection_2darray_updated.svg">

In [ ]:
# Create an array filled with random values
columns_test = np.random.random((5,5))

## Vector Math

In [ ]:
my_numbers = [[6,5], [9,1], [2,4], [7,14], [8,6]]

In [ ]:
sums = []
for row in my_numbers:
    row_sums = row[0] + row[1]
    sums.append(row_sums)
    
print(sums)

In [ ]:
# convert the list of lists to an ndarray
my_numbers = np.array(my_numbers)

<div class="alert alert-block alert-info">
<b>Vaja:</b> 
Use vector addition to add revenues (1) and profits (3). Assign the result to revenues_and_profits.</div>



<div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p>The following table lists the arithmetic operators implemented in NumPy:</p>
<table>
<thead><tr>
<th>Operator</th>
<th>Equivalent ufunc</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>+</code></td>
<td><code>np.add</code></td>
<td>Addition (e.g., <code>1 + 1 = 2</code>)</td>
</tr>
<tr>
<td><code>-</code></td>
<td><code>np.subtract</code></td>
<td>Subtraction (e.g., <code>3 - 2 = 1</code>)</td>
</tr>
<tr>
<td><code>-</code></td>
<td><code>np.negative</code></td>
<td>Unary negation (e.g., <code>-2</code>)</td>
</tr>
<tr>
<td><code>*</code></td>
<td><code>np.multiply</code></td>
<td>Multiplication (e.g., <code>2 * 3 = 6</code>)</td>
</tr>
<tr>
<td><code>/</code></td>
<td><code>np.divide</code></td>
<td>Division (e.g., <code>3 / 2 = 1.5</code>)</td>
</tr>
<tr>
<td><code>//</code></td>
<td><code>np.floor_divide</code></td>
<td>Floor division (e.g., <code>3 // 2 = 1</code>)</td>
</tr>
<tr>
<td><code>**</code></td>
<td><code>np.power</code></td>
<td>Exponentiation (e.g., <code>2 ** 3 = 8</code>)</td>
</tr>
<tr>
<td><code>%</code></td>
<td><code>np.mod</code></td>
<td>Modulus/remainder (e.g., <code>9 % 4 = 1</code>)</td>
</tr>
</tbody>
</table>

</div>
</div>


## Calculating Statistics For 1D ndarrays

<div class="alert alert-block alert-info">
<b>Vaja:</b> Use the ndarray.min() method to calculate the minimum value of revenues. Assign the result to revenues_min.
Use the ndarray.mean() method to calculate the average value of profits. Assign the result to profits_mean.</div>

<div>

<table>
<thead>
<tr>
<th>Calculation</th>
<th>Function Representation</th>
<th>Method Representation</th>
</tr>
</thead>
<tbody>
<tr>
<td>Calculate the minimum value of <code>trip_mph</code></td>
<td><code>np.min(trip_mph)</code></td>
<td><code>trip_mph.min()</code></td>
</tr>
<tr>
<td>Calculate the maximum value of <code>trip_mph</code></td>
<td><code>np.max(trip_mph)</code></td>
<td><code>trip_mph.max()</code></td>
</tr>
<tr>
<td>Calculate the <a target="_blank" href="https://en.wikipedia.org/wiki/Mean">mean average</a> value of <code>trip_mph</code></td>
<td><code>np.mean(trip_mph)</code></td>
<td><code>trip_mph.mean()</code></td>
</tr>
<tr>
<td>Calculate the <a target="_blank" href="https://en.wikipedia.org/wiki/Median">median average</a> value of <code>trip_mph</code></td>
<td><code>np.median(trip_mph)</code></td>
<td>There is no ndarray median method</td>
</tr>
</tbody>
</table>
</div>

## Calculating Statistics For 2D ndarrays

<img alt="Dimensional Arrays" src="./images/array_method_axis_none.svg">

<img alt="Dimensional Arrays" src="./images/array_method_axis_1.svg">

<img alt="Dimensional Arrays" src="./images/array_method_axis_0.svg">



<p><img alt="The axis parameter" src="https://s3.amazonaws.com/dq-content/289/axis_param.svg"></p>


## Datatypes

In [ ]:
x = np.array([1, 2])   # Let numpy choose the datatype
print(x.dtype)         # Prints "int64"
print(x.nbytes)

x = np.array([1.0, 2.0])   # Let numpy choose the datatype
print(x.dtype)
print(x.nbytes)

In [ ]:
x = np.array([1, 2], dtype=np.int32)   # Force a particular datatype
print(x.dtype)                         # Prints "int32"
print(x.nbytes)

x = np.array([1, 2], dtype=np.int8)   # Force a particular datatype
print(x.dtype)                         # Prints "int32"
print(x.nbytes)

<div class="text_cell_render border-box-sizing rendered_html">
<table>
<thead><tr>
<th>Data type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>bool_</code></td>
<td>Boolean (True or False) stored as a byte</td>
</tr>
<tr>
<td><code>int_</code></td>
<td>Default integer type (same as C <code>long</code>; normally either <code>int64</code> or <code>int32</code>)</td>
</tr>
<tr>
<td><code>intc</code></td>
<td>Identical to C <code>int</code> (normally <code>int32</code> or <code>int64</code>)</td>
</tr>
<tr>
<td><code>intp</code></td>
<td>Integer used for indexing (same as C <code>ssize_t</code>; normally either <code>int32</code> or <code>int64</code>)</td>
</tr>
<tr>
<td><code>int8</code></td>
<td>Byte (-128 to 127)</td>
</tr>
<tr>
<td><code>int16</code></td>
<td>Integer (-32768 to 32767)</td>
</tr>
<tr>
<td><code>int32</code></td>
<td>Integer (-2147483648 to 2147483647)</td>
</tr>
<tr>
<td><code>int64</code></td>
<td>Integer (-9223372036854775808 to 9223372036854775807)</td>
</tr>
<tr>
<td><code>uint8</code></td>
<td>Unsigned integer (0 to 255)</td>
</tr>
<tr>
<td><code>uint16</code></td>
<td>Unsigned integer (0 to 65535)</td>
</tr>
<tr>
<td><code>uint32</code></td>
<td>Unsigned integer (0 to 4294967295)</td>
</tr>
<tr>
<td><code>uint64</code></td>
<td>Unsigned integer (0 to 18446744073709551615)</td>
</tr>
<tr>
<td><code>float_</code></td>
<td>Shorthand for <code>float64</code>.</td>
</tr>
<tr>
<td><code>float16</code></td>
<td>Half precision float: sign bit, 5 bits exponent, 10 bits mantissa</td>
</tr>
<tr>
<td><code>float32</code></td>
<td>Single precision float: sign bit, 8 bits exponent, 23 bits mantissa</td>
</tr>
<tr>
<td><code>float64</code></td>
<td>Double precision float: sign bit, 11 bits exponent, 52 bits mantissa</td>
</tr>
<tr>
<td><code>complex_</code></td>
<td>Shorthand for <code>complex128</code>.</td>
</tr>
<tr>
<td><code>complex64</code></td>
<td>Complex number, represented by two 32-bit floats</td>
</tr>
<tr>
<td><code>complex128</code></td>
<td>Complex number, represented by two 64-bit floats</td>
</tr>
</tbody>
</table>

</div>

## Boolean Indexing

### Boolean Arrays

In [ ]:
print(type(3.5) == float)

In [ ]:
print(5 > 6)

In [ ]:
print(np.array([2,4,6,8]) + 10)

<div class="alert alert-block alert-info">
Use vectorized boolean operations to:
<li> Evaluate whether the elements in array a are less than 3. Assign the result to a_bool.</li> 
<li> Evaluate whether the elements in array b are equal to "blue". Assign the result to b_bool.</li> 
<li>  Evaluate whether the elements in array c are greater than 100. Assign the result to c_bool.</li> </div>

In [ ]:
a = np.array([1, 2, 3, 4, 5])
b = np.array(["blue", "blue", "red", "blue"])
c = np.array([80.0, 103.4, 96.9, 200.3])

### Boolean Indexing with 1D ndarrays

In [ ]:
c = np.array([80.0, 103.4, 6.9, 200.3])


<div class="alert alert-block alert-info">Izračunamo nekaj statistik za f500 podatke.</div>

### Boolean Indexing with 2D ndarrays

<img alt="Dimensional Arrays" src="./images/bool_dims_updated.svg">

## The meaning of shapes in NumPy

**1-DIMENSIONAL NUMPY ARRAYS ONLY HAVE ONE AXIS**

**2-DIMENSIONAL NUMPY ARRAYS**

In [ ]:
a = np.arange(12)
print(a)

<pre class="lang-py s-code-block"><code class="hljs language-python">┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│  <span class="hljs-number">0</span> │  <span class="hljs-number">1</span> │  <span class="hljs-number">2</span> │  <span class="hljs-number">3</span> │  <span class="hljs-number">4</span> │  <span class="hljs-number">5</span> │  <span class="hljs-number">6</span> │  <span class="hljs-number">7</span> │  <span class="hljs-number">8</span> │  <span class="hljs-number">9</span> │ <span class="hljs-number">10</span> │ <span class="hljs-number">11</span> │
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘
</code></pre>

In [ ]:
a.shape

In [ ]:
# lokacija podatkov v pomnilniku
a.__array_interface__['data']

<pre class="lang-py s-code-block"><code class="hljs language-python">i= <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">3</span>    <span class="hljs-number">4</span>    <span class="hljs-number">5</span>    <span class="hljs-number">6</span>    <span class="hljs-number">7</span>    <span class="hljs-number">8</span>    <span class="hljs-number">9</span>   <span class="hljs-number">10</span>   <span class="hljs-number">11</span>
┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│  <span class="hljs-number">0</span> │  <span class="hljs-number">1</span> │  <span class="hljs-number">2</span> │  <span class="hljs-number">3</span> │  <span class="hljs-number">4</span> │  <span class="hljs-number">5</span> │  <span class="hljs-number">6</span> │  <span class="hljs-number">7</span> │  <span class="hljs-number">8</span> │  <span class="hljs-number">9</span> │ <span class="hljs-number">10</span> │ <span class="hljs-number">11</span> │
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘
</code></pre>

In [ ]:
a[2]

In [ ]:
b.shape

In [ ]:
b.__array_interface__['data']

<pre class="lang-py s-code-block"><code class="hljs language-python">i= <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">1</span>    <span class="hljs-number">1</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">2</span>    <span class="hljs-number">2</span>    <span class="hljs-number">2</span>
j= <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">3</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">3</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">3</span>
┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│  <span class="hljs-number">0</span> │  <span class="hljs-number">1</span> │  <span class="hljs-number">2</span> │  <span class="hljs-number">3</span> │  <span class="hljs-number">4</span> │  <span class="hljs-number">5</span> │  <span class="hljs-number">6</span> │  <span class="hljs-number">7</span> │  <span class="hljs-number">8</span> │  <span class="hljs-number">9</span> │ <span class="hljs-number">10</span> │ <span class="hljs-number">11</span> │
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘
</code></pre>

<pre class="lang-py s-code-block"><code class="hljs language-python">i= <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>
j= <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">1</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">2</span>    <span class="hljs-number">2</span>    <span class="hljs-number">3</span>    <span class="hljs-number">3</span>    <span class="hljs-number">3</span>
┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│  <span class="hljs-number">0</span> │  <span class="hljs-number">1</span> │  <span class="hljs-number">2</span> │  <span class="hljs-number">3</span> │  <span class="hljs-number">4</span> │  <span class="hljs-number">5</span> │  <span class="hljs-number">6</span> │  <span class="hljs-number">7</span> │  <span class="hljs-number">8</span> │  <span class="hljs-number">9</span> │ <span class="hljs-number">10</span> │ <span class="hljs-number">11</span> │
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘
</code></pre>

<pre class="lang-py s-code-block"><code class="hljs language-python">i= <span class="hljs-number">0</span>    <span class="hljs-number">1</span>    <span class="hljs-number">2</span>    <span class="hljs-number">3</span>    <span class="hljs-number">4</span>    <span class="hljs-number">5</span>    <span class="hljs-number">6</span>    <span class="hljs-number">7</span>    <span class="hljs-number">8</span>    <span class="hljs-number">9</span>   <span class="hljs-number">10</span>   <span class="hljs-number">11</span>
j= <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>    <span class="hljs-number">0</span>
┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐
│  <span class="hljs-number">0</span> │  <span class="hljs-number">1</span> │  <span class="hljs-number">2</span> │  <span class="hljs-number">3</span> │  <span class="hljs-number">4</span> │  <span class="hljs-number">5</span> │  <span class="hljs-number">6</span> │  <span class="hljs-number">7</span> │  <span class="hljs-number">8</span> │  <span class="hljs-number">9</span> │ <span class="hljs-number">10</span> │ <span class="hljs-number">11</span> │
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘
</code></pre>

## Assigning Values

### Assigning Values in ndarrays

    ndarray[location_of_values] = new_value

In [ ]:
a = np.array(['red','blue','black','blue','purple'])
a[0] = 'orange'
print(a)

In [ ]:
ones = np.ones((3,5))


### Assignment Using Boolean Arrays

In [ ]:
a2 = np.array([1, 2, 3, 4, 5])
a2[a2 > 2] = 99

print(a2)

In [ ]:
b = np.linspace(1, 9, num=9, dtype=np.int32)


    bool = array[:, column_for_comparison] == value_for_comparison
    array[bool, column_for_assignment] = new_value

and then all in one line:

    array[array[:, column_for_comparison] == value_for_comparison, column_for_assignment] = new_value

## Adding Rows and Columns to ndarrays

In [ ]:
# primer 2 -1d
ones = np.ones(shape=3)
print(ones)
print(ones.shape)

In [ ]:
# primer 1 -1d
ones = np.ones(3)
print(ones)
print(ones.shape)

In [ ]:
# primer 3 -2d
ones = np.ones(shape=(3,1))
print(ones)
print("----------")
print(ones[0])
print("----------")
print(ones[0,0])
print(ones.shape)

In [ ]:
# primer 4 -2d
ones = np.ones(shape=(1,3))
print(ones)
print("----------")
print(ones[0])
print("----------")
print(ones[0,0])
print(ones.shape)

In [ ]:
# primer 5 -3d
ones = np.ones(shape=(2,3,2))
print(ones)
print("----------")
print(ones[0])
print("----------")
print(ones[0,0])
print(ones.shape)

In [ ]:
ones = np.ones((2, 3))
print(ones)

In [ ]:
zeros = np.zeros(3)
print(zeros)

## Computation on NumPy Arrays: Universal Functions


### The Slowness of Loops



In [ ]:
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

In [ ]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit -n 1 -r 1 compute_reciprocals(big_array)

### Introducing UFuncs (Universal functions)

[Docs](https://numpy.org/doc/stable/reference/ufuncs.html)



In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

In [ ]:
%timeit (1.0 / big_array)

## Subarrays as no-copy views

 

In [ ]:
r = np.ones((4,4))
print(r)

In [ ]:
r2 = r[:2,:2]
print(r2)

## Copying Data



In [ ]:
#To avoid this, use r.copy to create a copy that will not affect the original array
r3 = np.random.randint(10, size=(5, 5)) 
print(r3, '\n')

